In [43]:
import os 
import pandas as pd
from tqdm import tqdm
import numpy as np

from src.data_types import TOB, Order
from src.exchange import TOB_Exchange

%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# define folder path
# folder_path = 'C:/Users/gobel/code/research_env/keyrock/bi-repo/data/tardis/datasets/'
folder_path = '/Users/jan-gobeli/Code/bi-repo/data/tardis/datasets/'
# folder_path = '/Users/nick-kokkaliaris/Desktop/Keyrock/Keyrock/Tardis Playground/datasets/'
files_list = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'binance' 
    and f.split('_')[-1] == 'COMPBTC.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'quotes']
files_list

['binance_quotes_2023-07-02_COMPBTC.csv.gz',
 'binance_quotes_2023-07-05_COMPBTC.csv.gz',
 'binance_quotes_2023-07-04_COMPBTC.csv.gz',
 'binance_quotes_2023-07-03_COMPBTC.csv.gz']

In [3]:
bin_trades = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'binance' 
    and f.split('_')[-1] == 'COMPBTC.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'trades']
u_trades = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'upbit' 
    and f.split('_')[-1] == 'BTC-COMP.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'trades']
ufiles_list = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'upbit' 
    and f.split('_')[-1] == 'BTC-COMP.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'quotes']

In [4]:
def get_data(files_list):
    df = pd.DataFrame()
    # Use a loop to get all the csv.gz files from the path location in a single Dataframe
    for csv_file in tqdm(files_list):
        file_name = (folder_path + csv_file)
        if file_name[-2:] == "gz":
            df1 = pd.read_csv(file_name, compression='gzip')
            # Concatenate all the dataframes into a single dataframe
            df = pd.concat([df, df1], ignore_index=True)
    return df

In [5]:
upbit = get_data(ufiles_list)
binance = get_data(files_list)

upbit_t = get_data(u_trades)
binance_t = get_data(bin_trades)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 49.70it/s]


In [6]:
binance['time'] = pd.to_datetime(binance.timestamp, unit='us')
upbit['time'] = pd.to_datetime(upbit.timestamp, unit='us')
binance_t['time'] = pd.to_datetime(binance_t.timestamp, unit='us')
upbit_t['time'] = pd.to_datetime(upbit_t.timestamp, unit='us')

binance.set_index('time', inplace=True)
upbit.set_index('time', inplace=True)
binance_t.set_index('time', inplace=True)
upbit_t.set_index('time', inplace=True)

In [7]:
binance_t = binance_t[['timestamp', 'side', 'price', 'amount']].groupby(['timestamp', 'side', 'price']).sum().reset_index()

Updates: 
- 1: bid quantity update 
- 2: new top bid 
- 3: new top ask 
- 4: ask quantity update 
- 5: complete update

In [38]:
binance['latency'] = binance.local_timestamp - binance.timestamp

In [67]:
binance.latency.std() / binance.latency.mean()

0.2902754220552034

In [68]:
lst = binance[['timestamp', 'bid_amount', 'bid_price', 'ask_price', 'ask_amount']].sort_values('timestamp').to_numpy()

exchg = TOB_Exchange(latency=[5000, 0.29])
exchg.load_tob(lst, 'COMPBTC')

In [14]:
exchg.limit_order('COMPBTC', 20, 0.001712, 1)
exchg.limit_order('COMPBTC', 10, 0.001718, 0)

In [35]:
for i in range(5):
    exchg._simulation_step()

Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
